In [1]:
import numpy as np
#from sklearn.linear_model import Ridge
#from sklearn.linear_model import Lasso
#from sklearn.linear_model import LassoCV
import os
from tqdm import tnrange, tqdm_notebook
import matplotlib.pyplot as plt
import time
import scipy
from scipy import sparse
from celer import LassoCV


# FIX NEURAL MAKER

In [2]:
def new_make_neural_matrix(spike_train_file, image_times_file, save_dir):
    train_times = np.asarray(h5py.File(image_times_file, 'r')["image_times"])[0,:]
    test_times = np.asarray(h5py.File(image_times_file, 'r')["test_image_times"])
    print(test_times.shape)
    train_images = train_times.shape[0]
    
    spike_train = np.load(spike_train_file)
    print(spike_train.shape)
    spike_times = spike_train[:,0] / 20000
    units = np.max(spike_train[:,1]).astype(np.int) + 1
    
    interval_add = np.arange(0.01, 0.5, 0.01)
    
    train_matrix = np.zeros((train_images, units, 50))
    bin_edges = np.copy(train_times)
    
    for i in range(test_times.shape[1]):
        bin_edges = np.append(bin_edges, test_times[0,i])
        
    for i in tnrange(int(train_images)):
        for j in range(interval_add.shape[0]):
            bin_edge = train_times[i] + interval_add[j]
            bin_edges = np.append(bin_edges, bin_edge)
    
    bin_edges = np.append(bin_edges,9999)
    bin_edges = np.sort(bin_edges)
    print(bin_edges.shape)
    
    hist_spikes, hist_edges = np.histogram(spike_times, bin_edges)
    count = spike_times.shape[0] - np.sum(hist_spikes)
    final_bin = hist_spikes[-1]
    hist_spikes = hist_spikes[:-1]
    print(hist_spikes.shape)
    
    delete_ids = np.arange(50000,450009,50001)
    print(delete_ids)
    deleted = hist_spikes[delete_ids]
    
    hist_spikes = np.delete(hist_spikes, delete_ids)
    print(hist_spikes.shape)
    print(np.sum(hist_spikes) + np.sum(deleted) + final_bin + count)
    print(np.sum(hist_spikes))
    
    for i in tnrange(hist_spikes.shape[0]):
        bin_count = hist_spikes[i]
        for j in range(bin_count):  
                image = i//50
                unit = int(spike_train[count,1])
                index = i - 50*image
                train_matrix[image, unit, index] += 1
                count += 1
        if i%50000 == 49999 and i<499999:
            print(i)
            print(i//50000)
            count += deleted[i//50000]
    count += final_bin
    print(count)
    print(spike_times.shape)
    
    test_matrix = train_matrix[9900:,:,:]
    valid_matrix = train_matrix[9800:9900, :, :]
    train_matrix = train_matrix[:9800,:,:]
    
    print(np.sum(train_matrix) + np.sum(test_matrix) + np.sum(valid_matrix))
    
    np.save(os.path.join(save_dir, "yass_50_train_neural.npy"), train_matrix)
    np.save(os.path.join(save_dir, "yass_50_test_neural.npy"), test_matrix)
    np.save(os.path.join(save_dir, "yass_50_valid_neural.npy"), valid_matrix)
    
    

In [3]:
spike_train_file = "/ssd/joon/2017_11_29_ns/yass/yass_spiketrain.npy"
image_times_file = "/ssd/joon/2017_11_29_ns/trigger_times_201711290.mat"
save_dir = "/ssd/joon/2017_11_29_ns/yass/neural/"

new_make_neural_matrix(spike_train_file, image_times_file, save_dir)


(150, 10)
(134785693, 2)



(500011,)
(500009,)
[ 50000 100001 150002 200003 250004 300005 350006 400007 450008]
(500000,)
134785693
116506002


49999
0
99999
1
149999
2
199999
3
249999
4
299999
5
349999
6
399999
7
449999
8

134785693
(134785693,)
116506002.0


# Ridge (50 Time Bins)

In [2]:
def valid_ridge_reg(train_image_file, valid_image_file, train_neural_file, valid_neural_file, reg, index, save_dir):
    train_images = np.load(train_image_file).astype(np.float32)
    valid_images = np.load(valid_image_file)
    train_neural = np.load(train_neural_file).astype(np.float32)
    valid_neural = np.load(valid_neural_file)
    
    ridge = Ridge(alpha = reg)
    ridge.fit(train_neural, train_images)
    predict = ridge.predict(valid_neural)
    
    corr_sum = 0
    for i in range(predict.shape[1]):
        valid_pixels = valid_images[:,i]
        exp_pixels = predict[:,i]
        corr = np.corrcoef(valid_pixels, exp_pixels)[0][1]
        corr_sum += corr
    print(reg)
    print(corr_sum/predict.shape[1])
        
def multi_valid_ridge_reg(train_image_file, valid_image_file, train_neural_file, valid_neural_file, save_dir):
    reg_list = np.logspace(1, 4, 10, endpoint=True)
    for i in tnrange(len(reg_list)):
        reg = reg_list[i]
        valid_ridge_reg(train_image_file, valid_image_file, train_neural_file, valid_neural_file, reg, i, save_dir)
    
def test_ridge_reg(train_image_file, test_image_file, train_neural_file, test_neural_file, reg, dec_save, weight_save):
    train_images = np.load(train_image_file).astype(np.float32)
    test_images = np.load(test_image_file).astype(np.float32)
    train_neural = np.load(train_neural_file).astype(np.float32)
    test_neural = np.load(test_neural_file).astype(np.float32)
    
    ridge = Ridge(alpha = reg)
    ridge.fit(train_neural, train_images)
    print(ridge.coef_.shape)
    predict = ridge.predict(test_neural)
    
    corr_sum = 0
    for i in range(predict.shape[1]):
        test_pixels = test_images[:,i]
        exp_pixels = predict[:,i]
        corr = np.corrcoef(test_pixels, exp_pixels)[0][1]
        corr_sum += corr
    print(reg)
    print(corr_sum/predict.shape[1])
    
    np.save(os.path.join(dec_save, "yass_50_ridge_smooth_test_decoded.npy"), predict)
    np.save(os.path.join(weight_save, "yass_50_ridge_weights.npy"), ridge.coef_)
    

In [3]:
train_image_file = "/ssd/joon/2017_11_29_ns/images/smooth_train_images.npy"
valid_image_file = "/ssd/joon/2017_11_29_ns/images/smooth_valid_images.npy"
train_neural_file = "/ssd/joon/2017_11_29_ns/yass/neural/yass_lin_50_train_neural.npy"
valid_neural_file = "/ssd/joon/2017_11_29_ns/yass/neural/yass_lin_50_valid_neural.npy"
save_dir = "/ssd/joon/2017_11_29_ns/yass/decoded/"



multi_valid_ridge_reg(train_image_file, valid_image_file, train_neural_file, valid_neural_file, save_dir)

MemoryError: Unable to allocate array with shape (13870, 104700) and data type float32

In [3]:
train_image_file = "/ssd/joon/2017_11_29_ns/images/smooth_train_images.npy"
train_neural_file = "/ssd/joon/2017_11_29_ns/yass/neural/yass_lin_50_train_neural.npy"
test_image_file = "/ssd/joon/2017_11_29_ns/images/smooth_test_images.npy"
test_neural_file = "/ssd/joon/2017_11_29_ns/yass/neural/yass_lin_50_test_neural.npy"
reg = 464.158883
dec_save= "/ssd/joon/2017_11_29_ns/yass/decoded/"
weight_save = "/ssd/joon/2017_11_29_ns/yass/weights/"


test_ridge_reg(train_image_file, test_image_file, train_neural_file, test_neural_file, reg, dec_save, weight_save)

(13870, 104700)
464.158883
0.9790201759103617


# LASSO

In [3]:
def valid_lasso_reg(train_images_file, valid_images_file, train_neural_file, valid_neural_file, save_dir):
    
    train_images = np.load(train_images_file)
    valid_images = np.load(valid_images_file)
    train_neural = np.load(train_neural_file)
    valid_neural = np.load(valid_neural_file)
    
    train_neural = sparse.coo_matrix(train_neural).tocsc()
    valid_neural = sparse.coo_matrix(valid_neural).tocsc()
    
    image_no = train_images.shape[0]
    pix_no = train_images.shape[1]
    feat_no = train_neural.shape[1]
    
    corr_sum = 0
    
    weights = np.empty((feat_no, pix_no))
    decoded = np.empty((valid_images.shape[0], pix_no))
    
    ids = np.arange(pix_no)
    np.random.shuffle(ids)
    
    bad_count = 0
    
    for i in tnrange(pix_no):

        train_pix = train_images[:,i]
        
        alpha_list = np.logspace(-3,0,6)
        
        lasso = LassoCV(random_state = 0, n_jobs=-1, alphas=alpha_list, selection="random", cv=3)
        lasso.fit(train_neural, train_pix)
        predict = lasso.predict(valid_neural)
        
        print(lasso.alpha_)
        
        weights[:,i] = lasso.coef_
        decoded[:,i] = predict
        
        if lasso.alpha_ == 0.1 or lasso.alpha_ == 1:
            print(i, lasso.alpha_)
            bad_count += 1
        
        valid_pix = valid_images[:,i]
        corr = np.corrcoef(valid_pix.reshape((-1,)), predict.reshape((-1,)))[0][1]
        corr_sum += corr
        
        print(corr)
        
        if i%500 == 499:
            np.save(os.path.join(save_dir, "yass_l1_bin_0_to_"+str(i)+"_weights.npy"), weights[:,:i+1])
            np.save(os.path.join(save_dir, "yass_l1_bin_0_to_"+str(i)+"_smooth_decoded.npy"), decoded[:,:i+1])
            print(weights[:,:i+1].shape)
            print(decoded[:,:i+1].shape)

    print(corr_sum/pix_no)
    print(bad_count)

    np.save(os.path.join(save_dir, "yass_l1_bin_smooth_weights.npy"), weights)
    np.save(os.path.join(save_dir, "yass_l1_bin_smooth_decoded.npy"), decoded)


In [4]:
train_images_file = "/ssd/joon/2017_11_29_ns/images/smooth_train_images.npy"
#train_neural_file = "/ssd/joon/2017_11_29_ns/yass/neural/yass_train_neural.npy"
valid_images_file = "/ssd/joon/2017_11_29_ns/images/smooth_test_images.npy"
#valid_neural_file = "/ssd/joon/2017_11_29_ns/yass/neural/yass_test_neural.npy"

#train_neural_file = "/ssd/joon/2017_11_29_ns/yass/neural/yass_lin_50_train_neural.npy"
#valid_neural_file = "/ssd/joon/2017_11_29_ns/yass/neural/yass_lin_50_test_neural.npy"

save_dir = "/ssd/joon/2017_11_29_ns/yass/l1_bin_partial/"


valid_lasso_reg(train_images_file, valid_images_file, train_neural_file, valid_neural_file, save_dir)

0.0630957344480193
0.9879866898474345


KeyboardInterrupt: 

In [2]:
print(np.logspace(-3,0,6))

[0.001      0.00398107 0.01584893 0.06309573 0.25118864 1.        ]


## Global Lambda Test

In [2]:
def test_lasso_reg(train_images_file, test_images_file, train_neural_file, test_neural_file, reg, save_dir):
    
    train_images = np.load(train_images_file).astype(np.float32)[:,:4500]
    test_images = np.load(test_images_file).astype(np.float32)[:,:4500]
    train_neural = np.load(train_neural_file).astype(np.float32)
    test_neural = np.load(test_neural_file).astype(np.float32)
    
    print(train_images.shape)
    print(test_images.shape)
    print(train_neural.shape)
    print(test_neural.shape)
    
    image_no = train_images.shape[0]
    pix_no = train_images.shape[1]
    feat_no = train_neural.shape[1]
    
    corr_sum = 0
    
    weights = np.empty((feat_no, pix_no))
    decoded = np.empty((test_images.shape[0], pix_no))
    
    print(weights.shape)
    print(decoded.shape)
    
    for i in tnrange(4500):
        train_pix = train_images[:,i]
        lasso = Lasso(alpha=reg, selection="random")
        lasso.fit(train_neural, train_pix)
        predict = lasso.predict(test_neural)
        decoded[:,i] = predict
        weights[:,i] = lasso.coef_
        
        test_pix = test_images[:,i]
        corr = np.corrcoef(test_pix.reshape((-1,)), predict.reshape((-1,)))[0][1]
        corr_sum += corr
        if i%500 == 499:
            np.save(os.path.join(save_dir, "yass_l1_0_to_"+str(i)+"_weights.npy"), weights[:,:i+1])
            np.save(os.path.join(save_dir, "yass_l1_0_to_"+str(i)+"_smooth_decoded.npy"), decoded[:,:i+1])
        
    np.save(os.path.join(save_dir, "yass_l1_0_to_4500_FINAL_weights.npy"), weights)
    np.save(os.path.join(save_dir, "yass_l1_0_to_4500_FINAL_smooth_decoded.npy"), decoded)
    
        

In [3]:
train_images_file = "/ssd/joon/2017_11_29_ns/images/smooth_train_images.npy"
train_neural_file = "/ssd/joon/2017_11_29_ns/yass/neural/yass_lin_50_train_neural.npy"
test_images_file = "/ssd/joon/2017_11_29_ns/images/smooth_test_images.npy"
test_neural_file = "/ssd/joon/2017_11_29_ns/yass/neural/yass_lin_50_test_neural.npy"
save_dir = "/ssd/joon/2017_11_29_ns/yass/l1_partial/"
reg = 0.85




test_lasso_reg(train_images_file, test_images_file, train_neural_file, test_neural_file, reg, save_dir)

KeyboardInterrupt: 

KeyboardInterrupt: 

# Reduce Neural and Images

In [4]:
neural = np.load("/ssd/joon/2017_11_29_ns/yass/neural/yass_lin_50_valid_neural.npy")[:,:]
images = np.load("/ssd/joon/2017_11_29_ns/images/smooth_valid_images.npy")[:,:146*5]
save_dir = "/ssd/joon/2017_11_29_ns/yass/high_pass/"

np.save(os.path.join(save_dir, "hp_valid_neural.npy"), neural)
np.save(os.path.join(save_dir, "hp_smooth_valid_images.npy"), images)

# LASSO CELER

In [2]:
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning) 

def celer_lassocv_reg(train_images_file, test_images_file, train_neural_file, test_neural_file, save_dir):
    train_images = np.load(train_images_file)
    test_images = np.load(test_images_file)
    train_neural = np.load(train_neural_file)[:,:-1]
    test_neural = np.load(test_neural_file)[:,:-1]
    
    train_neural = sparse.coo_matrix(train_neural).tocsc()
    test_neural = sparse.coo_matrix(test_neural).tocsc()

    image_no = train_images.shape[0]
    pix_no = train_images.shape[1]
    feat_no = train_neural.shape[1]
    
    weights = np.empty((feat_no, pix_no))
    decoded = np.empty((100, pix_no))
    
    print(weights.shape)
    print(decoded.shape)
    
    corr_sum = 0
    
    bad_count = 0
    
    for i in tnrange(4000, pix_no):
        time_1 = time.time()
        
        train_pix = train_images[:,i]
        #alpha_list = np.logspace(-3.5,-1.5,20)
        #alpha_list = np.logspace(-3,-2,10)
        
        alpha_list = np.logspace(-3.1, -2.1,10)
        
        celer_lasso = LassoCV(alphas=alpha_list, cv=3, n_jobs=-1, normalize=True, prune=True, tol=10)
        celer_lasso.fit(train_neural, train_pix)
        predict = celer_lasso.predict(test_neural)
        
        if celer_lasso.alpha_ > 0.0078 or celer_lasso.alpha_ < 0.0008:
            print(i, "bad!")
            bad_count += 1
        
        weights[:,i] = celer_lasso.coef_
        decoded[:,i] = predict
        
        test_pix = test_images[:,i]
        corr = np.corrcoef(test_pix, predict)[0][1]
        print(corr)
        
        corr_sum += corr
        
        if i%500 == 499:
            np.save(os.path.join(save_dir, "yass_celer_l1_bin_4000_"+str(i)+"_weights.npy"), weights[:,4000:i+1])
            np.save(os.path.join(save_dir, "yass_celer_l1_bin_4000_"+str(i)+"_smooth_decoded.npy"), decoded[:,4000:i+1])
        time_2 = time.time()
        
    print(corr_sum/pix_no)
    print(bad_count)

    np.save(os.path.join(save_dir, "yass_celer_l1_bin_4000_i_smooth_weights.npy"), weights[:,4000:])
    np.save(os.path.join(save_dir, "yass_celer_l1_bin_4000_i_smooth_decoded.npy"), decoded[:,4000:])



In [ ]:
train_images_file = "/ssd/joon/2017_11_29_ns/images/smooth_train_images.npy"
test_images_file = "/ssd/joon/2017_11_29_ns/images/smooth_test_images.npy"
train_neural_file = "/ssd/joon/2017_11_29_ns/yass/neural/yass_train_neural.npy"
test_neural_file = "/ssd/joon/2017_11_29_ns/yass/neural/yass_test_neural.npy"
save_dir = "/ssd/joon/2017_11_29_ns/yass/celer_partial/"



celer_lassocv_reg(train_images_file, test_images_file, train_neural_file, test_neural_file, save_dir)

(4188, 13870)
(100, 13870)


0.9854293910970159
0.9846396712212995
0.9854797986324288
0.9844671524806159
0.9846270617606878
0.9850554036898032
0.984790642762816
0.98559441748252
0.9850994925965649
0.9853153883869469
0.9834666026433287
0.9829678985613168
0.9839430785153616
0.9833406577546687
0.9818895690153948
0.9803428207508759
0.9791920428565735
0.9791655370274108
0.9775479571694371
0.9770693097364882
0.9762639089032342
0.9760851889474905
0.9760128693659181
0.9772323182099731
0.9777954665507057
4025 bad!
0.9765969234578141
0.9792245223112969
0.9797538109649682
0.9810442924790882
0.9805354387630618
0.9816183588471606
0.9819876862978401
0.980182699316804
0.9806036154635938
0.9807336606942035
0.9842294077474877
0.984175725080559
0.9828368445358505
0.9815681278632359
0.9774220222917972
0.9755718067207866
0.9743731747291677
0.9722035662891555
0.9693419414846354
0.970459812928625
0.9712357472939902
0.9728714581538298
0.9750191801038451
0.9772228403006914
0.9787823613827085
0.9798405950199234
0.9821595045244033
0.983120

4402 bad!
0.9753110887126941
0.9767097823859621
0.9773159181415404
4405 bad!
0.9777395197342799
4406 bad!
0.979472930191076
4407 bad!
0.9807099829449379
4408 bad!
0.9821271013134575
0.9840501926532995
0.9847795195161763
0.9862984713766098
4412 bad!
0.9850038160474065
0.9863503655898908
0.9879635203625231
0.9882518400827491
4416 bad!
0.9869801477997978
0.9885118751854773
0.9886411056444614
0.9885942308258432
0.9883511229427067
0.9906794161262495
0.9906270946278366
0.9908240388186167
0.9910436545253168
0.9899421543211182
0.9894749169892647
0.9890227431326553
0.9884875505438538
0.9882859075264028
0.9869311834401094
0.9846505428843725
0.9843590528315347
4433 bad!
0.982741992604664
0.9857191736895593
0.985230327698485
0.9852283731979469
0.9850180893908727
0.9846963536117166
0.9846455257907671
0.9845417183586053
0.983358713905552
0.9827227017948221
0.981089617174044
0.9812265593097904
0.978897537564096
0.9802353845050081
0.9764955892759425
0.976161406268123
0.9755607707751174
0.9752588774898

0.9797569382547044
0.9781068423023133
0.9783853149091827
0.9779336649850664
0.9763691662241571
0.9756802773967647
0.975923918914452
0.9751595588492937
0.9751333791043336
0.974019173255943
0.9730895296846681
0.9713025942325906
0.9692601713002569
0.9662647491265368
0.9639584939506715
0.9619338642357729
0.960807158829616
0.9814416212274979
0.9804947010630791
0.9801103521756833
0.9799770773190514
0.9803632536083495
0.9804820651729469
0.979638096395457
0.97914396785384
0.9787449286649608
0.9778028589552071
0.977941104830573
0.9793390765349286
0.9793378224481472
4831 bad!
0.9799800090077433
0.9831822960087132
4833 bad!
0.9820573501344039
4834 bad!
0.9818282835101934
4835 bad!
0.9815829422682747
0.9819307812566701
4837 bad!
0.9800406036613014
4838 bad!
0.9793190148665766
4839 bad!
0.97905525945931
4840 bad!
0.9795919864442281
4841 bad!
0.9800782576774116
0.9819921805250654
0.9839773407117883
4844 bad!
0.9830731244197886
0.9853218978023118
4846 bad!
0.9846915002256759
4847 bad!
0.9851239977844

0.9807962321184674
0.9822205269438913
0.9829608393902928
0.9847193318966719
0.9846643859147814
0.9848313400754979
0.9856656168348796
0.9849948345039695
0.9841652470298481
0.9833287858583543
0.9819235844236716
0.9811449083223799
0.9804007963680764
0.9800931888997391
0.9803072061296795
0.9802393033295423
0.9818018575132634
0.9831159119720775
0.984928847507414
0.9853859386654409
0.9860088285624468
0.9863870665206049
0.9856155254298157
0.9851769708420725
0.9848453423459902
0.9840158969562134
0.9845865817833688
0.9833914512767208
0.9834227761259866
0.982477679706349
0.9827347963469025
0.9820874451604645
0.9824319245488331
0.9817455773765449
0.98144262820649
0.9805536795438958
0.9803705309959516
0.9793727843179755
0.9791516554118713
0.9786064192060014
0.9781529326369967
0.9775940952600968
0.9779506278150286
0.9765821366970729
0.9769676311923948
0.9765022819385081
0.9761658548038723
0.9760202130064517
0.9754471833078112
0.9743852598768454
0.9731712210647241
0.9697082303687662
0.96626116237069

0.9771062177117452
0.9784397072320649
5597 bad!
0.9748505742165446
0.9766716013907004
5599 bad!
0.9749153831382502
5600 bad!
0.9748881945500055
0.9762727669134544
5602 bad!
0.9754509796555363
5603 bad!
0.9759417832063536
5604 bad!
0.9757056732532237
0.9772287114430747
5606 bad!
0.9757902277930658
5607 bad!
0.9758497355164171
0.977166713381499
0.9770053485244441
5610 bad!
0.9751017572779601
5611 bad!
0.97515232374558
0.9776996446581093
5613 bad!
0.9763727556388035
0.9795088543425713
0.9814646136426747
5616 bad!
0.9769879920854543
5617 bad!
0.9768097334621335
0.9774271852047476
0.9769282896268185
0.9758926288402636
5621 bad!
0.9731164252859771
0.9741167180019007
0.9739017785580903
5624 bad!
0.9721559746487367
0.9744912720792719
0.9758488115054551
0.9777738798318356
0.9795447515077654
0.9798406588835162
0.9802883775047321
0.9810821778975648
0.9815972518785313
0.9802785949395718
0.9808085156098733
0.9790139895269563
0.9790135557718155
0.9794240896615528
0.9792441734937457
0.978078571825668